In [14]:
import pandas as pd
import math
file = "points.xlsx"
data = pd.read_excel(file)
# 1. Count the number of games played
games_played = data.groupby('Player')['Date'].count()

# 2. Count the number of games won
games_won = data[data['Team'] == data['Winning Team']].groupby('Player')['Date'].count()

# 3. Calculate the win ratio
win_ratio = games_won / games_played

# 4. Sum the number of penalties
penalties = data.groupby('Player')['Penalty'].sum()

# 5. Sum the number of friend referrals
friend_referrals = data.groupby('Player')['Friend Referrals'].sum()

# 6. Sum the number of own goals
own_goals = data.groupby('Player')['Own Goals'].sum()

# 7. Sum the MVP points (using MVP, not MVP Points)
mvp_points = data[data['MVP'] == 1].groupby('Player').size()

# 8. Sum the SPL Bonus points (using SPL Bonus, not SPL Bonus Points)
spl_bonus = data[data['SPL Bonus'] == 1].groupby('Player').size()

# 9. Sum the goals conceded
goals_conceded = data.groupby('Player')['Goals Conceded'].sum()

# 10. Calculate the average goals per game
avg_goals = data.groupby('Player')['Goals'].mean()

# 11. Sum the total number of goals
total_goals = data.groupby('Player')['Goals'].sum()

# 12. Calculate the average total points per game
avg_total_points = data.groupby('Player')['Total Points'].mean()

# Combine the above metrics into a single dataframe
summary = pd.DataFrame({
    'Games Played': games_played,
    'Games Won': games_won,
    'Win Ratio': win_ratio,
    'Penalties': penalties,
    'Friend Referrals': friend_referrals,
    'Own Goals': own_goals,
    'MVP': mvp_points,
    'SPL Bonus': spl_bonus,
    'Goals Conceded': goals_conceded,
    'Average Goals/Game': avg_goals,
    'Total Goals': total_goals,
    'Average Total Points/Game': avg_total_points
})

# Fill NaN values with 0 (for players who might not have won a game or received MVP points or SPL Bonus points)
summary = summary.fillna(0)

# 13. Rank the players based on the sum of their Total points
summary['Total Points'] = data.groupby('Player')['Total Points'].sum()
summary = summary.sort_values(by='Total Points', ascending=False)


# 1. Calculate cumulative Total Points for each player after each game
data['Cumulative Points'] = data.groupby('Player')['Total Points'].cumsum()

# 2. Determine the player's rank based on these cumulative points after each game
data['Rank'] = data.groupby('Date')['Cumulative Points'].rank(method="first", ascending=False)

# Sort data to ensure we process in chronological order for each player
data_sorted = data.sort_values(by=['Player', 'Date'])

# 3. Calculate the change in rank between each game for every player
data_sorted['Rank Change'] = data_sorted.groupby('Player')['Rank'].diff().fillna(0)

# Extract the latest rank change for each player
latest_rank_change = data_sorted.groupby('Player').apply(lambda x: x.iloc[-1])['Rank Change']

# Add this column to the summary table
summary['Rank Change'] = latest_rank_change

# Convert columns to integer type as instructed
cols_to_int = ['Games Won', 'MVP', 'SPL Bonus', 'Rank Change']
summary[cols_to_int] = summary[cols_to_int].astype(int)
# Calculate the overall rank based on the 'Total Points'
summary['Overall Rank'] = summary['Total Points'].rank(method="min", ascending=False).astype(int)

# Reorder columns to have 'Overall Rank' at the front
column_order = ['Overall Rank'] + [col for col in summary if col != 'Overall Rank']
summary = summary[column_order]

summary


,Overall Rank,Games Played,Games Won,Win Ratio,Penalties,Friend Referrals,Own Goals,MVP,SPL Bonus,Goals Conceded,Average Goals/Game,Total Goals,Average Total Points/Game,Total Points,Rank Change
Player,,,,,,,,,,,,,,,
Daniele (Francesco),1,17,12,0.705882,0,1,0,0,0,108,3.588235,61,14.470588,246,0
Luca Stoppi,2,23,8,0.347826,1,4,2,1,1,166,0.782609,18,10.652174,245,1
Andrea Limonta,3,22,11,0.500000,0,2,1,0,1,154,0.136364,3,10.500000,231,0
Cameron McAinsh,4,22,9,0.409091,0,1,0,0,0,181,1.818182,40,10.318182,227,-1
Damiano Barbanera,5,21,7,0.333333,0,2,0,0,1,161,2.285714,48,10.571429,222,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dario (Francesco),59,1,0,0.000000,0,0,0,0,0,9,1.000000,1,6.000000,6,0
Puddi,59,1,0,0.000000,0,0,0,0,0,9,0.000000,0,6.000000,6,0
Ale (Fabri),62,1,0,0.000000,0,0,0,0,0,10,0.000000,0,5.000000,5,0
